# Agent-Based Modeling and Simulation

In this lecture, we will study how to leverage "Agent"-based simulation to understand complex questions. An agent-based model (ABM) is a computational model for simulating the actions and interactions of autonomous agents (both individual or collective entities such as organizations or groups) in order to understand the behavior of a system and what governs its outcomes. We will show how Python programming can be used to build simple, but effective agent-based simulators.

**Grocery Store Checkout Line Optimization** You’re at the grocery store rolling a cart filled with all the essentials like pizza bagels, ice cream and hummus. Now all you want to do after a long day is get home to watch the latest episode of “Game of Thrones.” As you approach the checkout area, your eyes widen with horror. Every single line snakes back through the aisles. Other irritated shoppers are restlessly eyeing the other lines to see which one is the shortest—which only makes you more antsy. We want to understand how stores can optimize checkout lines by allocating resources to regular lines or "express" lines (e.g., 10 items or less). This seems like a challenging problem, but we will try to break it down into smaller pieces that we can understand.

## Recipe for ABM Problems 
There are three key questions when starting with any ABM problem. 
* **What** Write down what do we want to discover with the ABM problem.
* **Who** Write down all of the key "agents" in the system (entities that take actions)
* **How** Write down all of the key formulas and distributiosn that govern agent actions

How does this work in our grocery store problem?
* **What** "I want a model to show that replacing a regular line with an express line can improve average customer satisfaction by reducing the average waiting time"
* **Who** There are three key agents in the problem: Store, Users, and Lines (a proxy for checkout employees).
* **How** We will use a D/U/K queueing model to describe customer arrivals and service, a Poisson distribution model for describing customer basket sizes, and a dissatisfaction model for customers where dissatisfaction is directly proportional to waiting time divided by expected service time. 

## Programming our Agents
Now that we know the basics of our problem, let's actually code the agents. First, let's start with our "Store" Agent. The "store" will manage the entire simulation and maintain the key parameters.

In [1]:
import numpy as np

class Store: 
    '''The Store class provides the basic wrapper and logic for
       the simulation used in this lecture
    '''
    
    #constructor
    def __init__(self, 
                 num_regular_lines, #Number of regular lines
                 num_express_lines, #Number of express lines
                 express_threshold, #Threshold of items in the express lines
                 arrival_rate, #Arrival rate (in customers per tick) < 1 (int)
                 service_rate, #Service rate (in items scanned per tick)
                 avg_items_per_cust, #Avg items per customer
                 seed = 0):
    
        #store the parameters
        self.num_regular_lines = num_regular_lines
        self.num_express_lines = num_express_lines
        self.express_threshold = express_threshold
        self.rec_arrival_rate = int(1.0/arrival_rate)
        self.service_rate = service_rate
        self.avg_items_per_cust = avg_items_per_cust
        self.seed = seed
        
        #manage the simulation state
        self.lines = [Line(service_rate)]*num_regular_lines + \
                     [Line(service_rate,express_threshold)]*num_express_lines
        
        self.completed_users = [] 
        
        self.clock = 0
        
        np.random.seed(seed)
        
    
    #runs one time step of the simulation
    def tick(self):
        
        
        #new users arrive in a deterministic queuing model periodically on the tick
        if self.clock % self.rec_arrival_rate == 0:
            
            u = User(self.clock, self.avg_items_per_cust)
            
            
            #find the best line to join (what does this code do?)
            line_list = [ (u.items > l.item_limit, l.total_items(), i) for i,l in enumerate(self.lines)]
            line_list.sort()
            
            self.lines[line_list[0][2]].users.append(u)
        
        
        #run checkouts
        for l in self.lines:
            u = l.tick()
            
            if not (u is None):
                self.completed_users.append(u)
                
        self.clock += 1

Note how ABM simulations have a recursive structure. The "store" agent's action at each step triggers  the user and line actions.

In [2]:
class Line:
    '''The line class models a checkout line that services items with a uniform service rate
    '''
    
    def __init__(self, service_rate, item_limit = float("inf")):
        self.service_rate = service_rate
        self.item_limit = item_limit
        
        #state
        self.users = []
    
    def length(self):
        return len(self.users)
    
    def tick(self):
        if len(self.users) == 0:
            return None
        
        #why does this work
        items_checked_out = np.random.rand(1)*self.service_rate #checks out a fractional item
        
        self.users[0].scan(items_checked_out)
        for i in range(1, len(self.users)):
            self.users[i].wait_in_line()
        
        if self.users[0].done():
            return self.users.pop(0)
        else:
            return None
    
    def total_items(self):
        return sum([u.items for u in self.users])

In [3]:
class User:
    '''The user class models an individual customer in the store
    '''
    
    def __init__(self, arrival_time, avg_items_per_customer):
        self.arrival_time = arrival_time
        
        self.basket_size = max(np.random.poisson(avg_items_per_customer),1) 
        #simulate the basket size with a poisson model
        
        #state
        self.wait_time = 0
        self.service_time = 0
        self.items = self.basket_size
    
    def scan(self, items_checked_out): #checks out a fractional item
        self.service_time += 1
        self.items = max(self.items - items_checked_out, 0)
        
        
    def wait_in_line(self):
        self.wait_time += 1
    
    def done(self):
        return (self.items <= 0)
    
    def __str__(self):
        return "User arrived at: " + str(self.arrival_time) + " and bought " + str(self.basket_size) + \
               " items and waited for " + str(self.wait_time + self.service_time) + " ticks \n"
    
    __repr__ = __str__

## Making Inferences
Now, we are ready to use our models to make inferences about resource allocation in grocery stores. Let's start off with some data.

* The average customer in a grocery store purchases 8.14 items per visit
* The average employee can service a line (scan/bag) at the rate of 1 item every 10 secs
* At a peak hour, a customer will enter  the store every 30 secs

We're going to use this to simplify our coding a little bit. Let's define a "typical store" as:

In [4]:
def TypicalStore(num_regular_lines,  num_express_lines, express_threshold):
    return Store(num_regular_lines = num_regular_lines, #Number of regular lines
                 num_express_lines = num_express_lines, #Number of express lines
                 express_threshold = express_threshold, #Threshold of items in the express lines
                 arrival_rate = 0.03, #Arrival rate (in customers per tick) < 1 (int)
                 service_rate = 0.1, #Service rate (in items scanned per tick)
                 avg_items_per_cust = 8.14)

Now let's define some key metrics

In [5]:
def avg_time_in_store(store):
    return np.mean([u.wait_time + u.service_time for u in store.completed_users])

def avg_dissatisfaction(store):
    return np.mean([(u.wait_time + u.service_time)/(u.basket_size * store.service_rate) for u in store.completed_users])

In [6]:
s = TypicalStore(5,1,10)

for i in range(100000):
    s.tick()
    
print('Time Spent: ',avg_time_in_store(s))
print('Dissatisfaction: ', avg_dissatisfaction(s))

Time Spent:  181.417107001321
Dissatisfaction:  225.81363219899856


In [7]:
# Num line versus dissatisfaction
for lines in range(1,10):  
    s = TypicalStore(lines,1,10)
    
    for i in range(1000):
        s.tick()
        
    print('Regular Lines',lines,'Time Spent: ',avg_time_in_store(s))
    print('Regular Lines',lines,'Dissatisfaction: ', avg_dissatisfaction(s))

Regular Lines 1 Time Spent:  401.1
Regular Lines 1 Dissatisfaction:  500.8253968253968
Regular Lines 2 Time Spent:  453.1764705882353
Regular Lines 2 Dissatisfaction:  602.8039215686274
Regular Lines 3 Time Spent:  438.3809523809524
Regular Lines 3 Dissatisfaction:  697.0342712842713
Regular Lines 4 Time Spent:  204.25925925925927
Regular Lines 4 Dissatisfaction:  260.06770081770077
Regular Lines 5 Time Spent:  143.68965517241378
Regular Lines 5 Dissatisfaction:  206.8331716176544
Regular Lines 6 Time Spent:  165.58620689655172
Regular Lines 6 Dissatisfaction:  203.79816639299398
Regular Lines 7 Time Spent:  162.24137931034483
Regular Lines 7 Dissatisfaction:  201.24799721351448
Regular Lines 8 Time Spent:  183.3
Regular Lines 8 Dissatisfaction:  202.99948292448292
Regular Lines 9 Time Spent:  159.73333333333332
Regular Lines 9 Dissatisfaction:  202.58973063973065


In [8]:
# Num line versus dissatisfaction
for lines in range(0,5):  
    s = TypicalStore(6-lines,lines,10)
    
    for i in range(10000):
        s.tick()
        
    print('Express Lines',lines,'Time Spent: ',avg_time_in_store(s))
    print('Express Lines',lines,'Dissatisfaction: ', avg_dissatisfaction(s))

Express Lines 0 Time Spent:  190.87128712871288
Express Lines 0 Dissatisfaction:  237.6246184600145
Express Lines 1 Time Spent:  172.6158940397351
Express Lines 1 Dissatisfaction:  222.20340049294077
Express Lines 2 Time Spent:  197.36212624584718
Express Lines 2 Dissatisfaction:  239.3328944034924
Express Lines 3 Time Spent:  216.53311258278146
Express Lines 3 Dissatisfaction:  261.92117635426945
Express Lines 4 Time Spent:  222.61129568106313
Express Lines 4 Dissatisfaction:  267.86384748510994


So that doesn't seem to make a difference (or at least it isn't apparent)! Let's dig a bit deeper into these results. 

## Q1. Determine the Simulation Uncertainty
Your first task is to determine how much the simulation varies run to run. This will help us understand the significance of differences between different settings. Let's assume a store with 5 regular lines, 1 express line, and a 10 item threshold. Let's consider a collection of 1000 tick simulations of this store. Determine the standard deviation of the average time in store and the average dissatisfaction. Hint: think about the random seed.

In [9]:
avg_t = []
avg_d = []

for i in range(1000):
    s = Store(num_regular_lines = 5, #Number of regular lines
                 num_express_lines = 1, #Number of express lines
                 express_threshold = 10, #Threshold of items in the express lines
                 arrival_rate = 0.03, #Arrival rate (in customers per tick) < 1 (int)
                 service_rate = 0.1, #Service rate (in items scanned per tick)
                 avg_items_per_cust = 8.14,
                 seed = i)
    
    for j in range(1000):
        s.tick()
    
    avg_t.append(avg_time_in_store(s))
    avg_d.append(avg_dissatisfaction(s))

print('STD (Average Time):', np.std(avg_t))
print('STD (Average Dissatisfaction):', np.std(avg_d))

STD (Average Time): 21.7345220491911
STD (Average Dissatisfaction): 15.915382484348358


## Q2.  Are Express Lines Worth It?
Now let's make a slightly busier store where customers come in 15% more frequently.

In [10]:
def BusyStore(num_regular_lines,  num_express_lines, express_threshold):
    return Store(num_regular_lines = num_regular_lines, #Number of regular lines
                 num_express_lines = num_express_lines, #Number of express lines
                 express_threshold = express_threshold, #Threshold of items in the express lines
                 arrival_rate = 0.035, #Arrival rate (in customers per tick) < 1 (int)
                 service_rate = 0.1, #Service rate (in items scanned per tick)
                 avg_items_per_cust = 8.14)

Write code that compares a store with 5 regular lines, 1 express line, and a 10 item threshold; and one with 6 regular lines. What do your results show?

In [11]:
s = BusyStore(5, 1, 10)

for i in range(1000):
    s.tick()
    
print('Regular Lines 5 Time Spent: ',avg_time_in_store(s))
print('Regular Lines 5 Dissatisfaction: ', avg_dissatisfaction(s))

s = BusyStore(6, 0, 10)

for i in range(1000):
    s.tick()

print('Regular Lines 6 Time Spent: ',avg_time_in_store(s))
print('Regular Lines 6 Dissatisfaction: ', avg_dissatisfaction(s))

Regular Lines 5 Time Spent:  184.36363636363637
Regular Lines 5 Dissatisfaction:  238.17759092759093
Regular Lines 6 Time Spent:  481.20588235294116
Regular Lines 6 Dissatisfaction:  685.2299465240641


The **time spent** and the **dissatisfaction level** in the store with 5 regular lines and 1 express line are drastically smaller than those in the store with 6 regular lines. The difference between times spent is \~297 and the difference between the dissatisfaction levels is \~447. These differences can be attributed to turning a single regular line into an express line.

## Q3. Determine the Optimal Express Threshold
For the problem above, write code that determines the optimal express item threshold for a store with 5 regular lines, 1 express line.

In [12]:
dis = []

for i in range(0, 11):
    s = BusyStore(5, 1, i)
    
    for j in range(1000):
        s.tick()
        
    dis.append((avg_dissatisfaction(s), i))

optimal_threshold = min(dis)[1]
print(optimal_threshold)

10


## Q4. Queueing Instability
"Instability" in queue is where the line grows in  an uncontrolled way -- meaning there are not enough lines to service the incoming customers. Write code that determines the maximum arrival rate a typical store can sustain before hitting instability.

In [13]:
prev = 0

for i in range(90):
    s = Store(num_regular_lines = 5, #Number of regular lines
                 num_express_lines = 1, #Number of express lines
                 express_threshold = 10, #Threshold of items in the express lines
                 arrival_rate = 0.03 + (i * 0.01), #Arrival rate (in customers per tick) < 1 (int)
                 service_rate = 0.1, #Service rate (in items scanned per tick)
                 avg_items_per_cust = 8.14)
    for j in range(1000):
        s.tick()
    avg = avg_time_in_store(s)
    if (avg != prev):
        prev = avg
    else:
        print('Maximum arrival rate before instability:', (0.03 + (i * 0.01)))
        break

Maximum arrival rate before instability: 0.14


## Q5.  Experience predictability
We found that adding more than 5 lines to our typical store had diminishing returns in terms of reducing user dissatisfaction. Now, we want you to show that adding more lines improves the predictability (all users have similar dissatisfactions) of the user experience---even if the mean value stops improving.

In [14]:
dissat_list1 = []
dissat_list2 = []

for lines in range(1,1000):  
    s = TypicalStore(lines,1,10)
    
    for i in range(1000):
        s.tick()
    
    if (lines < 6):
        dissat_list1.append(avg_dissatisfaction(s))
    
    dissat_list2.append(avg_dissatisfaction(s))
        

variance_first_5 = (np.std(dissat_list1)) ** 2
variance_first_1000 = (np.std(dissat_list2)) ** 2

print('Variance for the first 5:', variance_first_5)
print('Variance for the first 1000:', variance_first_1000)

Variance for the first 5: 36420.17464609928
Variance for the first 1000: 495.2472621075018


As we add more lines, we can see that the variance for average dissatisfactions get significantly smaller. For the first 5 numbers of lines: 36420 vs. for the first 1000 numbers of lines: 495. Thus, the dissatisfaction level becomes more predictable as we add more lines to the Store.

# Q6. Customer Strategies
Write code the compares average customer satisfaction under different line choosing strategies.
* Always pick the express lane if you are eligible, then pick the others at random
* Pick an eligible line with the fewest people in front of you (instead of the total items)
* Randomly pick an eligible line.

Hint: you may have to modify the Store, Line, and/or User class. To answer these questions.

In [15]:
import numpy as np
import random

class Store: 
    ''' Store modfication for Stratedy #1
    '''
    
    #constructor
    def __init__(self, 
                 num_regular_lines, #Number of regular lines
                 num_express_lines, #Number of express lines
                 express_threshold, #Threshold of items in the express lines
                 arrival_rate, #Arrival rate (in customers per tick) < 1 (int)
                 service_rate, #Service rate (in items scanned per tick)
                 avg_items_per_cust, #Avg items per customer
                 seed = 0):
    
        #store the parameters
        self.num_regular_lines = num_regular_lines
        self.num_express_lines = num_express_lines
        self.express_threshold = express_threshold
        self.rec_arrival_rate = int(1.0/arrival_rate)
        self.service_rate = service_rate
        self.avg_items_per_cust = avg_items_per_cust
        self.seed = seed
        
        #manage the simulation state
        self.lines = [Line(service_rate)]*num_regular_lines + \
                     [Line(service_rate,express_threshold)]*num_express_lines
        
        self.completed_users = [] 
        
        self.clock = 0
        
        np.random.seed(seed)
        
    
    #runs one time step of the simulation
    def tick(self):
        
        
        #new users arrive in a deterministic queuing model periodically on the tick
        if self.clock % self.rec_arrival_rate == 0:
            
            u = User(self.clock, self.avg_items_per_cust)
            
            
            #users get in express line if possible, otherwise choose random
            express_lines = [ (u.items <= l.item_limit, l.total_items(), i) for i, l in enumerate(self.lines)]
            
            if express_lines:
                self.lines[express_lines[0][2]].users.append(u)
            else:
                line_list = [(u.items > l.item_limit, l.total_items(), i) for i,l in enumerate(self.lines)]
                regular_line_list = list(filter(lambda x: x[0] == False, line_list))
                self.lines[random.choice(regular_line_list)[2]].users.append(u)
        
        #run checkouts
        for l in self.lines:
            u = l.tick()
            
            if not (u is None):
                self.completed_users.append(u)
                
        self.clock += 1
        
s = TypicalStore(5,1,10)

for i in range(1000):
    s.tick()
    
print('Average Dissatisfaction (Strategy #1):', avg_dissatisfaction(s))

class Store: 
    ''' Store modfication for Stratedy #2
    '''
    
    #constructor
    def __init__(self, 
                 num_regular_lines, #Number of regular lines
                 num_express_lines, #Number of express lines
                 express_threshold, #Threshold of items in the express lines
                 arrival_rate, #Arrival rate (in customers per tick) < 1 (int)
                 service_rate, #Service rate (in items scanned per tick)
                 avg_items_per_cust, #Avg items per customer
                 seed = 0):
    
        #store the parameters
        self.num_regular_lines = num_regular_lines
        self.num_express_lines = num_express_lines
        self.express_threshold = express_threshold
        self.rec_arrival_rate = int(1.0/arrival_rate)
        self.service_rate = service_rate
        self.avg_items_per_cust = avg_items_per_cust
        self.seed = seed
        
        #manage the simulation state
        self.lines = [Line(service_rate)]*num_regular_lines + \
                     [Line(service_rate,express_threshold)]*num_express_lines
        
        self.completed_users = [] 
        
        self.clock = 0
        
        np.random.seed(seed)
        
    
    #runs one time step of the simulation
    def tick(self):
        
        
        #new users arrive in a deterministic queuing model periodically on the tick
        if self.clock % self.rec_arrival_rate == 0:
            
            u = User(self.clock, self.avg_items_per_cust)
            
            #users get in possible line with the fewest people            
            line_list = [(len(l.users), u.items > l.item_limit, i) for i, l in enumerate(self.lines)]
            line_list = list(filter(lambda x: x[1] == False, line_list))
            line_list.sort()
            self.lines[line_list[0][2]].users.append(u)
        
        #run checkouts
        for l in self.lines:
            u = l.tick()
            
            if not (u is None):
                self.completed_users.append(u)
                
        self.clock += 1
        
s = TypicalStore(5,1,10)

for i in range(1000):
    s.tick()
    
print('Average Dissatisfaction (Strategy #2):', avg_dissatisfaction(s))

class Store: 
    ''' Store modfication for Stratedy #3
    '''
    
    #constructor
    def __init__(self, 
                 num_regular_lines, #Number of regular lines
                 num_express_lines, #Number of express lines
                 express_threshold, #Threshold of items in the express lines
                 arrival_rate, #Arrival rate (in customers per tick) < 1 (int)
                 service_rate, #Service rate (in items scanned per tick)
                 avg_items_per_cust, #Avg items per customer
                 seed = 0):
    
        #store the parameters
        self.num_regular_lines = num_regular_lines
        self.num_express_lines = num_express_lines
        self.express_threshold = express_threshold
        self.rec_arrival_rate = int(1.0/arrival_rate)
        self.service_rate = service_rate
        self.avg_items_per_cust = avg_items_per_cust
        self.seed = seed
        
        #manage the simulation state
        self.lines = [Line(service_rate)]*num_regular_lines + \
                     [Line(service_rate,express_threshold)]*num_express_lines
        
        self.completed_users = [] 
        
        self.clock = 0
        
        np.random.seed(seed)
        
    
    #runs one time step of the simulation
    def tick(self):
        
        
        #new users arrive in a deterministic queuing model periodically on the tick
        if self.clock % self.rec_arrival_rate == 0:
            
            u = User(self.clock, self.avg_items_per_cust)
            
            #users randomly choose an eligible line            
            line_list = [(len(l.users), u.items > l.item_limit, i) for i, l in enumerate(self.lines)]
            line_list = list(filter(lambda x: x[1] == False, line_list))
            self.lines[random.choice(line_list)[2]].users.append(u)
        
        #run checkouts
        for l in self.lines:
            u = l.tick()
            
            if not (u is None):
                self.completed_users.append(u)
                
        self.clock += 1
        
s = TypicalStore(5,1,10)

for i in range(1000):
    s.tick()
    
print('Average Dissatisfaction (Strategy #3):', avg_dissatisfaction(s))

Average Dissatisfaction (Strategy #1): 379.0120827448414
Average Dissatisfaction (Strategy #2): 206.8331716176544
Average Dissatisfaction (Strategy #3): 265.12741951535054
